In [1]:
import numpy as np
#import tensorflow as tf
from glob import glob
import os

from os.path import join,basename
import json
import random
import pickle
import cv2 as cv
import pandas as pd
from natsort import natsorted
%matplotlib inline

In [2]:
random.seed(123)
np.random.seed(123)

In [3]:
src_dires={'co':join('dataset_raw','smarthome_raw','alphapose','CO'),
'co_json':join('dataset_raw','smarthome_raw','alphapose','CO','json'),
'co_kp_pkl':join('dataset_raw','smarthome_raw','alphapose','CO','kp_pkl'),
'co_bbox_pkl':join('dataset_raw','smarthome_raw','alphapose','CO','bbox_pkl')}

In [4]:
pose_target_dires={'co':join('datasets_features','smarthome','alphapose','CO'),
'cd':join('datasets_features','smarthome','CD')
}

In [5]:
co_classnames=natsorted(os.listdir(src_dires['co_bbox_pkl']))

In [6]:
limit_coarse={'Drink':196,

'Eat': 144,

'Pour':91}

In [8]:
seq_len=42

Scaling the Kps from -1 to 1

In [9]:
def get_scaled_kp(kp):
    kp=np.array(kp).reshape(17,3)
    kp=kp[:,:2]
    df=pd.DataFrame(kp,columns=['X','Y'])
    df=df.apply(lambda x: (x - x.min()) / (x.max() - x.min())*2-1)  
    return np.array(df).reshape(34) 

    
    
         

Continue from here evening

In [10]:
def get_kps_imgid(kp_pkl):
   features=[0]*seq_len
   selected_frames=[0]*seq_len
   with open(kp_pkl,'rb') as f:
      kp_infos=pickle.load(f)

   kps=list(kp_infos.values())
   img_ids=list(kp_infos.keys())

   


   total_frames=len(kps)

   skip_frames=total_frames//seq_len
 

   

   if skip_frames==0:
      padd_arr=np.zeros(34,dtype=np.float64)
      
      for i in range(seq_len):

         if i < total_frames:

            scaled_kp=get_scaled_kp(kps[i])

            features[i]=scaled_kp
            selected_frames[i]=img_ids[i]

         elif (i >=total_frames) and (i < seq_len):

            features[i]=padd_arr
            selected_frames[i]='None'
         
      features=np.array(features)
      return (selected_frames,features)

   elif skip_frames==1:

         selected_index=sorted(random.sample(range(total_frames),seq_len))

         features=np.array([get_scaled_kp(kps[i]) for i in selected_index])

         selected_frames=np.array([img_ids[i] for i in selected_index])

         return (selected_frames,features)

   else :
         

         selected_index=list(range(0,total_frames,skip_frames))
         if len(selected_index)>seq_len:
            np.random.shuffle(selected_index)
            
            selected_index=selected_index[:seq_len]
            selected_index=sorted(selected_index)

            features=np.array([get_scaled_kp(kps[i]) for i in selected_index])
            selected_frames=np.array([img_ids[i] for i in selected_index])
            return (selected_frames,features)
         
         elif len(selected_index)==seq_len:

            features=np.array([get_scaled_kp(kps[i]) for i in selected_index])
            selected_frames=np.array([img_ids[i] for i in selected_index])
            return (selected_frames,features)





     


    

In [11]:
features=[]
labels=[]
for idx,classname in enumerate(co_classnames):
    
    coarse_label=classname.split('.')[0]
    kps_dires=glob(join(src_dires['co_kp_pkl'],classname)+'/*.pickle')
    random.shuffle(kps_dires)
    limit=limit_coarse[coarse_label]
    kps_dires=kps_dires[:limit]

    total_samples=len(kps_dires)
    train_len=np.ceil(0.6*total_samples)
    val_len=np.ceil(0.20*total_samples)
    test_len=total_samples-train_len-val_len

    for idx,kp_dire in enumerate(kps_dires):
        filename=basename(kp_dire)[:-7]
        frame_id,feature=get_kps_imgid(kp_dire)

        features.append(feature)
        labels.append(idx)

        if idx <= train_len:

            saved_imgid_folder_dire=join(pose_target_dires['co'],'img_id','train',classname)

            saved_kps_folder_dire=join(pose_target_dires['co'],'kps','train',classname)

        elif idx >train_len and idx <=(train_len+val_len):

            saved_imgid_folder_dire=join(pose_target_dires['co'],'img_id','val',classname)
            saved_kps_folder_dire=join(pose_target_dires['co'],'kps','val',classname)

        else:

            saved_imgid_folder_dire=join(pose_target_dires['co'],'img_id','test',classname)
            saved_kps_folder_dire=join(pose_target_dires['co'],'kps','test',classname)

            



        if not os.path.exists(saved_imgid_folder_dire):
            os.makedirs(saved_imgid_folder_dire)

        if not os.path.exists(saved_kps_folder_dire):
            os.makedirs(saved_kps_folder_dire)


        saved_imgid_file_dire=saved_imgid_folder_dire+'/{}.npy'.format(filename)

        saved_kps_file_dire=saved_kps_folder_dire+'/{}.npy'.format(filename)

        np.save(saved_imgid_file_dire,frame_id)

        np.save(saved_kps_file_dire,feature)


    
features=np.array(features)
labels=np.array(labels)



         
    

: 

In [12]:
features.shape

(918, 25, 34)

In [12]:
from sklearn.model_selection import train_test_split



In [13]:
train_features, test_features, train_labels, test_labels=train_test_split(features, labels, test_size=0.25, random_state=101,stratify=labels)

ValueError: The test_size = 156 should be greater or equal to the number of classes = 208

In [14]:

saved_dires_train=join(pose_target_dires['co'],'features','train')
if not os.path.exists(saved_dires_train):
    os.makedirs(saved_dires_train)

saved_dires_test=join(pose_target_dires['co'],'features','test')
if not os.path.exists(saved_dires_test):
    os.makedirs(saved_dires_test)

In [15]:
np.save(join(saved_dires_train,'features.npy'),train_features)
np.save(join(saved_dires_train,'labels.npy'),train_labels)

np.save(join(saved_dires_test,'features.npy'),test_features)
np.save(join(saved_dires_test,'labels.npy'),test_labels)

: 

In [53]:
labels=np.load(join(saved_dires,'labels.npy'))

In [9]:
np.unique(labels)

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18])

In [9]:
x_train=np.load(join(saved_dires_test,'features.npy'))

In [10]:
x_train.shape

(474, 32, 34)

: 

In [7]:
classNames=os.listdir(paths['cs_train'])

In [53]:
test_dires=glob(join(paths['cs_train'],classNames[0])+'/*.json')

In [ ]:
test_dires

In [54]:
a=features_extraction(test_dires[0])

In [57]:
a

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0]

In [19]:
for json_dire in test_dires:
    features=[0]*seq_len
    with open(json_dire) as f:
        data=json.load(f)
    total_frames=len(data['frames'])
    skip_frames=total_frames//seq_len

    #print(skip_frames)

    if skip_frames==0:

        print(0)

        padd_arr=np.zeros(39,dtype=np.float64)

        for i in range(seq_len):

            if i < seq_len:

                features[i]=data['frames'][i][0]['pose3d']

            if (i >= seq_len) and (i < total_frames):

                features[i]=padd_arr
            
            #return features


    elif skip_frames==1:

        print(1)

        selected_frame=sorted(random.sample(range(total_frames),seq_len))

        features=np.array([data['frames'][i][0]['pose3d'] for i in selected_frame])

        #return features


    else :

        print(2)

        selected_frame=[i for i in range(0,total_frames,skip_frames)]
        if len(selected_frame)>seq_len:
            np.random.shuffle(selected_frame)
            
            selected_frame=selected_frame[:seq_len]
            selected_frame=sorted(selected_frame)

        f=np.array([data['frames'][i][0]['pose3d'] for i in selected_frame])
        
        #return features



        


   
    

2
2


IndexError: list index out of range

In [25]:
selected_frame

[0,
 11,
 22,
 33,
 44,
 55,
 66,
 77,
 110,
 121,
 132,
 143,
 154,
 165,
 176,
 187,
 198,
 209,
 220,
 231,
 242,
 253,
 264,
 275,
 286,
 297,
 308,
 319,
 330,
 341,
 352,
 363,
 374,
 385,
 396,
 407,
 418,
 429,
 440,
 451,
 462,
 473,
 484,
 495,
 506,
 517,
 528,
 539,
 550,
 561,
 572,
 583,
 594,
 605,
 616,
 627,
 638,
 649,
 660,
 671,
 682,
 693,
 704,
 715]

In [40]:
for i in  selected_frame:
        #print(i)

        try:
                data['frames'][i][0]['pose3d']
        except:
                print(i)

660


In [43]:
len(data['frames'][660])

0

In [47]:
len(data['frames'])

709